#### Comparing the gene signatures across three fetal stem cells populations and LGR5+/- cells 
- **Developed by:** Anna Maguza
- **Affilation:** Faculty of Medicine, Würzburg University
- **Date of creation:** 10th September 2024
- **Last modified date:** 10th September 2024

This notebook is created to compare three fetal stem cells populations and LGR5+/- cells to identify are there some shared gene signatures.
It contains from three parts:
* Extraction of LGR5+/- signatures (DEGs) from Haber, 2017 dataset and scoring them to fetal stem cells. In this part the common set of genes was obtained using macrogenes derived from `SATURN`.
* Using the list of Lgr5 intestinal stem cell signatures created in Muñoz, 2012 dataset and scoring them to fetal stem cells. In this part the common set of genes was obtained using `ensembl`orthologues.
* Compare and score signatures of adult ISCs subsets identified in Biton, 2018 to fetal stem cells. In this part the common set of genes was obtained using `ensembl`orthologues. 
+ Score the most highly expressed genes from Grün, 2016 LGR5+ cells to fetal stem cells. In this part the common set of genes was obtained using `ensembl`orthologues. 


+ Import packages

In [1]:
import scanpy as sc
import pandas as pd
import numpy as np

+ Load fetal cells

In [2]:
fetal_adata = sc.read_h5ad("data/Fetal_cells_filtered_raw.h5ad")

+ Normalize fetal cells

In [3]:
fetal_adata_log = fetal_adata.copy()
sc.pp.normalize_total(fetal_adata_log, target_sum = 1e6, exclude_highly_expressed = True) #1e6 because it is 10X
sc.pp.log1p(fetal_adata_log)

In [4]:
fetal_adata_log = fetal_adata_log[fetal_adata_log.obs['Cell States'].isin(['MTRNR2L12+ASS1+_SC', 'TA', 'RPS10+_RPS17+_SC',
                                                                           'FXYD3+_CKB+_SC'])]

/home/amaguza/miniforge3/envs/scanpy_env/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


In [5]:
fetal_adata_log_copy = fetal_adata_log.copy()

### Score signatures from Haber, 2017 cells

* Load mouse cells

In [5]:
mouse_adata = sc.read_h5ad("data/Haber_2017_Smartseq_stem_cells_remapped.h5ad")

/home/amaguza/miniforge3/envs/scanpy_env/lib/python3.11/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


+ Normalize dataset

In [6]:
mouse_adata_log = mouse_adata.copy()
sc.pp.normalize_total(mouse_adata_log, target_sum = 1e4, exclude_highly_expressed = True) #1e4 because it is SmartSeq2
sc.pp.log1p(mouse_adata_log)

* Create common set of genes from macrogenes

In [7]:
orthologues = pd.read_csv("data/saturn_results/macrogenes_derived_human_mouse_orthologues_10K.csv")

In [8]:
orthologues = orthologues.drop_duplicates(subset=['Top Mouse Gene', 'Top Human Gene'])

In [9]:
human_genes_in_ortho = set(orthologues['Top Human Gene'])
mouse_genes_in_ortho = set(orthologues['Top Mouse Gene'])

In [10]:
fetal_adata_log = fetal_adata_log[:, ~fetal_adata_log.var_names.duplicated()].copy()
fetal_adata_log = fetal_adata_log[:, fetal_adata_log.var_names.isin(human_genes_in_ortho)].copy()

/home/amaguza/miniforge3/envs/scanpy_env/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


In [11]:
fetal_adata_log

AnnData object with n_obs × n_vars = 231646 × 6798
    obs: 'Sample_ID', 'Cell Type', 'Study_name', 'Donor_ID', 'Diagnosis', 'Age', 'Region code', 'Fraction', 'Sex', 'Library_Preparation_Protocol', 'batch', 'Age_group', 'Location', 'Cell States', 'Cell States GCA', 'Chem', 'Layer', 'Cell States Kong', 'dataset', 'n_genes_by_counts', 'total_counts', 'total_counts_mito', 'pct_counts_mito', 'total_counts_ribo', 'pct_counts_ribo', 'Cell_ID', '_scvi_batch', '_scvi_labels', 'n_genes', 'n_counts'
    var: 'feature_types-0-0-0', 'gene_name-1-0-0', 'gene_id-0-0', 'GENE-1-0', 'n_counts', 'n_cells'
    uns: 'log1p'

In [12]:
mouse_adata_log = mouse_adata_log[:, ~mouse_adata_log.var_names.isin(['nan'])].copy()
mouse_adata_log = mouse_adata_log[:, ~mouse_adata_log.var_names.duplicated()].copy()
mouse_adata_log = mouse_adata_log[:, mouse_adata_log.var_names.isin(mouse_genes_in_ortho)].copy()

/home/amaguza/miniforge3/envs/scanpy_env/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/amaguza/miniforge3/envs/scanpy_env/lib/python3.11/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/amaguza/miniforge3/envs/scanpy_env/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


In [13]:
mouse_to_human = dict(zip(orthologues['Top Mouse Gene'], orthologues['Top Human Gene']))

In [14]:
mouse_adata_log.var['human_gene'] = mouse_adata_log.var_names.map(mouse_to_human)
mouse_adata_log.var_names = mouse_adata_log.var['human_gene']
mouse_adata_log = mouse_adata_log[:, ~mouse_adata_log.var_names.duplicated()].copy()

/home/amaguza/miniforge3/envs/scanpy_env/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


In [15]:
common_genes = set(fetal_adata_log.var_names) & set(mouse_adata_log.var_names)
fetal_adata_log = fetal_adata_log[:, fetal_adata_log.var_names.isin(common_genes)].copy()
mouse_adata_log = mouse_adata_log[:, mouse_adata_log.var_names.isin(common_genes)].copy()

/home/amaguza/miniforge3/envs/scanpy_env/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


In [16]:
print(f"Number of genes in human dataset: {fetal_adata_log.n_vars}")
print(f"Number of genes in mouse dataset: {mouse_adata_log.n_vars}")
print(f"Number of common genes: {len(common_genes)}")

Number of genes in human dataset: 5066
Number of genes in mouse dataset: 5066
Number of common genes: 5066


* Extract signatures using DEGs

In [18]:
sc.tl.rank_genes_groups(mouse_adata_log, groupby="cell_type", method='wilcoxon', n_genes=100)
mouse_df = sc.get.rank_genes_groups_df(mouse_adata_log, group=None)

/home/amaguza/miniforge3/envs/scanpy_env/lib/python3.11/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


+ Score LGR5+ signatures to fetal stem cells

In [19]:
lgr5_positive_signature = mouse_df[mouse_df["group"] == 'LGR5+ stem cell']
lgr5_positive_signature = lgr5_positive_signature['names'].tolist()

In [20]:
sc.tl.score_genes(fetal_adata_log, gene_list=lgr5_positive_signature, score_name='lgr5_positive_score')

/home/amaguza/miniforge3/envs/scanpy_env/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/amaguza/miniforge3/envs/scanpy_env/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


In [ ]:
sc.set_figure_params(dpi=300, figsize=(10, 10))
sc.pl.violin(fetal_adata_log, keys='lgr5_positive_score', groupby='Cell States', stripplot=True, inner="quartile")

+ Score LGR5- signatures to fetal stem cells

In [24]:
lgr5_negative_signature = mouse_df[mouse_df["group"] == 'LGR5- stem cell']
lgr5_negative_signature = lgr5_negative_signature['names'].tolist()

In [25]:
sc.tl.score_genes(fetal_adata_log, gene_list=lgr5_negative_signature, score_name='lgr5_negative_score')

/home/amaguza/miniforge3/envs/scanpy_env/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/amaguza/miniforge3/envs/scanpy_env/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


In [ ]:
sc.set_figure_params(dpi=300, figsize=(10, 10))
sc.pl.violin(fetal_adata_log, keys='lgr5_negative_score', groupby='Cell States', stripplot=True, inner="quartile")

### Score LGR5+ signatures from Muñoz, 2012 paper

+ Create a common set of genes using `ensembl`orthologues database

In [27]:
orthologues_ensembl = pd.read_csv("database_data/ensembl_data/mouse_to_human_orthologues_ensembl.txt", sep = "\t")

In [29]:
munoz_signatures= pd.read_excel("data/Muñoz_2012/embj2012166-sup-0002.xls", sheet_name = "S12")

In [30]:
munoz_signatures = munoz_signatures.iloc[2:]

In [31]:
munoz_signatures = munoz_signatures.rename(columns = {munoz_signatures.columns[1]: 'Description'})

+ Score all genes from Muñoz, 2012 dataset

In [32]:
merged = pd.merge(munoz_signatures['The intestinal stem cell signature'], orthologues_ensembl, left_on='The intestinal stem cell signature', right_on='Gene name', how='left')

In [37]:
munoz_signatures_genes = set(merged['Human gene name'].dropna().unique())

In [38]:
len(munoz_signatures_genes)

454

In [35]:
fetal_adata_log = fetal_adata_log_copy.copy()

In [36]:
fetal_adata_log

AnnData object with n_obs × n_vars = 231646 × 25445
    obs: 'Sample_ID', 'Cell Type', 'Study_name', 'Donor_ID', 'Diagnosis', 'Age', 'Region code', 'Fraction', 'Sex', 'Library_Preparation_Protocol', 'batch', 'Age_group', 'Location', 'Cell States', 'Cell States GCA', 'Chem', 'Layer', 'Cell States Kong', 'dataset', 'n_genes_by_counts', 'total_counts', 'total_counts_mito', 'pct_counts_mito', 'total_counts_ribo', 'pct_counts_ribo', 'Cell_ID', '_scvi_batch', '_scvi_labels', 'n_genes', 'n_counts'
    var: 'feature_types-0-0-0', 'gene_name-1-0-0', 'gene_id-0-0', 'GENE-1-0', 'n_counts', 'n_cells'
    uns: 'log1p'

In [39]:
sc.tl.score_genes(fetal_adata_log, gene_list=munoz_signatures_genes, score_name='lgr5_positive_score')

/home/amaguza/miniforge3/envs/scanpy_env/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/amaguza/miniforge3/envs/scanpy_env/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


In [ ]:
sc.set_figure_params(dpi=300, figsize=(10, 10))
sc.pl.violin(fetal_adata_log, keys='lgr5_positive_score', groupby='Cell States', stripplot=True, inner="quartile")

+ Score genes that are enriched in both transcriptomics and proteomics profiles

In [42]:
munoz_signatures['Description'].value_counts()

Description
enriched in Mass Spec, enriched in either Agilent, Affymetrix or both    147
overlap Agilent Affymetrix                                               122
significant in Affy, enriched in Agilent                                 119
significant in Agilent, enriched in Affy                                  50
overlap Agilent Affymetrix MassSpec                                       33
significant in Affy, enriched in Agilent and MassSpec                     28
significant in Agilent, enriched in Affy and MassSpec                      5
significant in Affy, enriched in MassSpec                                  3
significant in Agilent, enriched in MassSpec                               3
Name: count, dtype: int64

In [43]:
munoz_signatures_transcriptomics_and_proteomics = munoz_signatures[munoz_signatures['Description'].str.contains("enriched in Mass Spec, enriched in either Agilent, Affymetrix or both")]

In [44]:
merged = pd.merge(munoz_signatures_transcriptomics_and_proteomics['The intestinal stem cell signature'], orthologues_ensembl, left_on='The intestinal stem cell signature', right_on='Gene name', how='left')

In [45]:
munoz_signatures_genes_transcriptomics_and_proteomics = set(merged['Human gene name'].dropna().unique())

In [46]:
len(munoz_signatures_genes_transcriptomics_and_proteomics)

135

In [47]:
sc.tl.score_genes(fetal_adata_log, gene_list=munoz_signatures_genes_transcriptomics_and_proteomics, score_name='lgr5_positive_score')

/home/amaguza/miniforge3/envs/scanpy_env/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/amaguza/miniforge3/envs/scanpy_env/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


In [ ]:
sc.set_figure_params(dpi=300, figsize=(10, 10))
sc.pl.violin(fetal_adata_log, keys='lgr5_positive_score', groupby='Cell States', stripplot=True, inner="quartile")

### Compare with ISC populations from Biton, 2018

In [59]:
orthologues_ensembl = pd.read_csv("database_data/ensembl_data/mouse_to_human_orthologues_ensembl.txt", sep = "\t")

In [50]:
isc_1 = pd.read_excel("data/Biton_2018/1-s2.0-S0092867418313163-mmc1.xlsx", sheet_name = "ISC-I")
isc_2 = pd.read_excel("data/Biton_2018/1-s2.0-S0092867418313163-mmc1.xlsx", sheet_name = "ISC-II")
isc_3 = pd.read_excel("data/Biton_2018/1-s2.0-S0092867418313163-mmc1.xlsx", sheet_name = "ISC-III")

In [51]:
isc_1.columns = isc_1.iloc[4]
isc_1 = isc_1.iloc[5:]

isc_2.columns = isc_2.iloc[4]
isc_2 = isc_2.iloc[5:]

isc_3.columns = isc_3.iloc[4]
isc_3 = isc_3.iloc[5:]

In [52]:
isc_datasets = [isc_1, isc_2, isc_3]
isc_human_sets = []

for isc_df in isc_datasets:
    merged = pd.merge(isc_df['Gene symbol'], orthologues_ensembl, left_on='Gene symbol', right_on='Gene name', how='left')
    human_orthologs = set(merged['Human gene name'].dropna().unique())
    isc_human_sets.append(human_orthologs)


In [53]:
len(isc_human_sets[0])

759

In [54]:
len(isc_human_sets[1])

718

In [55]:
len(isc_human_sets[2])

859

In [63]:
isc_1_genes = list(isc_human_sets[0])
isc_1_genes = isc_1_genes[:100]

isc_2_genes = list(isc_human_sets[1])
isc_2_genes = isc_2_genes[:100]

isc_3_genes = list(isc_human_sets[2])
isc_3_genes = isc_3_genes[:100]

In [56]:
fetal_adata_log = fetal_adata_log_copy.copy()
fetal_adata_log

AnnData object with n_obs × n_vars = 231646 × 25445
    obs: 'Sample_ID', 'Cell Type', 'Study_name', 'Donor_ID', 'Diagnosis', 'Age', 'Region code', 'Fraction', 'Sex', 'Library_Preparation_Protocol', 'batch', 'Age_group', 'Location', 'Cell States', 'Cell States GCA', 'Chem', 'Layer', 'Cell States Kong', 'dataset', 'n_genes_by_counts', 'total_counts', 'total_counts_mito', 'pct_counts_mito', 'total_counts_ribo', 'pct_counts_ribo', 'Cell_ID', '_scvi_batch', '_scvi_labels', 'n_genes', 'n_counts'
    var: 'feature_types-0-0-0', 'gene_name-1-0-0', 'gene_id-0-0', 'GENE-1-0', 'n_counts', 'n_cells'
    uns: 'log1p'

In [64]:
sc.tl.score_genes(fetal_adata_log, gene_list=isc_1_genes, score_name='ISC1_score')
sc.tl.score_genes(fetal_adata_log, gene_list=isc_2_genes, score_name='ISC2_score')
sc.tl.score_genes(fetal_adata_log, gene_list=isc_3_genes, score_name='ISC3_score')

/home/amaguza/miniforge3/envs/scanpy_env/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/amaguza/miniforge3/envs/scanpy_env/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


/home/amaguza/miniforge3/envs/scanpy_env/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/amaguza/miniforge3/envs/scanpy_env/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


/home/amaguza/miniforge3/envs/scanpy_env/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/amaguza/miniforge3/envs/scanpy_env/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


In [ ]:
sc.set_figure_params(dpi=300, figsize=(10, 10))
sc.pl.violin(fetal_adata_log, keys='ISC1_score', groupby='Cell States', stripplot=True, inner="quartile")

In [ ]:
sc.set_figure_params(dpi=300, figsize=(10, 10))
sc.pl.violin(fetal_adata_log, keys='ISC2_score', groupby='Cell States', stripplot=True, inner="quartile")

In [ ]:
sc.set_figure_params(dpi=300, figsize=(10, 10))
sc.pl.violin(fetal_adata_log, keys='ISC3_score', groupby='Cell States', stripplot=True, inner="quartile")

#### Score for Grün, 2016 signatures 

In [187]:
grun_2016 = sc.read_h5ad("/mnt/LaCIE/annaM/gut_project/raw_data/Grun_2016/Grün_2016_all_cells.h5ad")

In [188]:
grun_2016.obs['gsm_description'].value_counts() 

gsm_description
Extracted RNA 3-week CD24+ control cells, non-traced                   96
Extracted RNA 3-week traced Lgr5+ cells, CD24+ fraction                96
Extracted RNA 5-day traced Lgr5+ cells replicate 1                     96
Extracted RNA 5-day traced Lgr5+ cells replicate 2                     96
Extracted RNA 5-day traced Lgr5+ cells replicate 3                     96
Extracted RNA 5-day traced Lgr5+ cells replicate 4                     96
Extracted RNA 5-day traced Lgr5+ cells replicate 5                     96
Extracted RNA 8-week CD24+ control cells, non-traced replicate 1       96
Extracted RNA 8-week CD24+ control cells, non-traced replicate 2       96
Extracted RNA 8-week traced Lgr5+ cells                                96
Extracted RNA 8-week traced Lgr5+ cells, CD24+ fraction replicate 1    96
Extracted RNA 8-week traced Lgr5+ cells, CD24+ fraction replicate 2    96
Extracted RNA 8-week traced Lgr5+ cells, CD24+ fraction replicate 3    96
Name: count, dtype: in

In [189]:
grun_2016 = grun_2016[grun_2016.obs['gsm_description'].str.contains("Lgr5+")]

/home/amaguza/miniforge3/envs/scanpy_env/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


In [190]:
grun_2016.var['gene_name'] = grun_2016.var_names.str.split("__").str[0]
grun_2016.var['chromosome'] = grun_2016.var_names.str.split("__").str[1]

grun_2016.var.index = grun_2016.var['gene_name']
grun_2016.var.index.name = 'gene_id'

/tmp/ipykernel_294841/2476610837.py:1: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  grun_2016.var['gene_name'] = grun_2016.var_names.str.split("__").str[0]


In [191]:
mean_counts = grun_2016.X.mean(axis=0)
grun_2016.var['mean_counts'] = mean_counts

grun_genes = grun_2016.var.sort_values('mean_counts', ascending=False)

In [192]:
grun_genes = pd.DataFrame(grun_genes)

In [193]:
grun_genes = pd.merge(grun_genes['gene_name'], orthologues_ensembl, left_on='gene_name', right_on='Gene name', how='left')

In [194]:
grun_genes = grun_genes['Human gene name'].dropna().unique()

In [195]:
signature_genes = grun_genes[:100]

+ Score genes

In [196]:
fetal_adata_log = fetal_adata_log_copy.copy()

In [197]:
sc.tl.score_genes(fetal_adata_log, gene_list=signature_genes, score_name='LGR5+_score')

/home/amaguza/miniforge3/envs/scanpy_env/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/amaguza/miniforge3/envs/scanpy_env/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


In [ ]:
sc.set_figure_params(dpi=300, figsize=(10, 10))
sc.pl.violin(fetal_adata_log, keys='LGR5+_score', groupby='Cell States', stripplot=True, inner="quartile")

+ Score populations from different cell age

In [199]:
grun_2016_3_week = grun_2016[grun_2016.obs['gsm_description'].str.contains("3-week")]
grun_2016_5_day = grun_2016[grun_2016.obs['gsm_description'].str.contains("5-day")]
grun_2016_8_week = grun_2016[grun_2016.obs['gsm_description'].str.contains("8-week")]

/home/amaguza/miniforge3/envs/scanpy_env/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


In [200]:
mean_counts = grun_2016_3_week.X.mean(axis=0)
grun_2016_3_week.var['mean_counts'] = mean_counts
grun_2016_3_week = grun_2016_3_week.var.sort_values('mean_counts', ascending=False)

/tmp/ipykernel_294841/3895890168.py:2: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  grun_2016_3_week.var['mean_counts'] = mean_counts
/home/amaguza/miniforge3/envs/scanpy_env/lib/python3.11/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [201]:
mean_counts = grun_2016_5_day.X.mean(axis=0)
grun_2016_5_day.var['mean_counts'] = mean_counts
grun_2016_5_day = grun_2016_5_day.var.sort_values('mean_counts', ascending=False)

/tmp/ipykernel_294841/3086603238.py:2: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  grun_2016_5_day.var['mean_counts'] = mean_counts
/home/amaguza/miniforge3/envs/scanpy_env/lib/python3.11/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [202]:
mean_counts = grun_2016_8_week.X.mean(axis=0)
grun_2016_8_week.var['mean_counts'] = mean_counts
grun_2016_8_week = grun_2016_8_week.var.sort_values('mean_counts', ascending=False)

/tmp/ipykernel_294841/3971722715.py:2: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  grun_2016_8_week.var['mean_counts'] = mean_counts
/home/amaguza/miniforge3/envs/scanpy_env/lib/python3.11/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [ ]:
grun_genes_3_week = pd.merge(grun_2016_3_week['gene_name'], orthologues_ensembl, left_on='gene_name', right_on='Gene name', how='left')
grun_genes_3_week = grun_genes_3_week['Human gene name'].dropna().unique()
signature_genes3_week = grun_genes_3_week[:100]

sc.tl.score_genes(fetal_adata_log, gene_list=signature_genes3_week, score_name='LGR5+_score_3_weeks_cells')
sc.set_figure_params(dpi=300, figsize=(10, 10))
sc.pl.violin(fetal_adata_log, keys='LGR5+_score_3_weeks_cells', groupby='Cell States', stripplot=True, inner="quartile")

In [ ]:
grun_2016_5_day = pd.merge(grun_2016_5_day['gene_name'], orthologues_ensembl, left_on='gene_name', right_on='Gene name', how='left')
grun_genes_5_day = grun_2016_5_day['Human gene name'].dropna().unique()
signature_genes5_day = grun_genes_5_day[:100]

sc.tl.score_genes(fetal_adata_log, gene_list=signature_genes5_day, score_name='LGR5+_score_5_days_cells')
sc.set_figure_params(dpi=300, figsize=(10, 10))
sc.pl.violin(fetal_adata_log, keys='LGR5+_score_5_days_cells', groupby='Cell States', stripplot=True, inner="quartile")

In [ ]:
grun_2016_8_week = pd.merge(grun_2016_8_week['gene_name'], orthologues_ensembl, left_on='gene_name', right_on='Gene name', how='left')
grun_2016_8_week = grun_2016_8_week['Human gene name'].dropna().unique()
signature_genes8_week = grun_2016_8_week[:100]

sc.tl.score_genes(fetal_adata_log, gene_list=signature_genes8_week, score_name='LGR5+_score_8_weeks_cells')
sc.set_figure_params(dpi=300, figsize=(10, 10))
sc.pl.violin(fetal_adata_log, keys='LGR5+_score_8_weeks_cells', groupby='Cell States', stripplot=True, inner="quartile")

In [210]:
fetal_adata_log.obs

,Sample_ID,Cell Type,Study_name,Donor_ID,Diagnosis,Age,Region code,Fraction,Sex,Library_Preparation_Protocol,...,pct_counts_ribo,Cell_ID,_scvi_batch,_scvi_labels,n_genes,n_counts,LGR5+_score,LGR5+_score_3_weeks_cells,LGR5+_score_5_days_cells,LGR5+_score_8_weeks_cells
cell_id,,,,,,,,,,,,,,,,,,,,,
AAACCTGCATCTACGA-1-Human_colon_16S8159182,F72-FPIL-0-SC-1,Epithelial,"Elmentaite, 2021",F72,Fetal Healthy,16Wk,FPIL,SC,Female,10x 5' v1,...,41.668846,AAACCTGCATCTACGA-1-Human_colon_16S8159182,0,2,2965,11787.0,5.774283,5.966382,5.785009,5.666526
AAACGGGCAGGTTTCA-1-Human_colon_16S8159182,F72-FPIL-0-SC-1,Epithelial,"Elmentaite, 2021",F72,Fetal Healthy,16Wk,FPIL,SC,Female,10x 5' v1,...,26.177607,AAACGGGCAGGTTTCA-1-Human_colon_16S8159182,0,2,1789,5313.0,5.731415,5.801635,5.758357,5.566543
AAACGGGTCGGCGCAT-1-Human_colon_16S8159182,F72-FPIL-0-SC-1,Epithelial,"Elmentaite, 2021",F72,Fetal Healthy,16Wk,FPIL,SC,Female,10x 5' v1,...,46.285980,AAACGGGTCGGCGCAT-1-Human_colon_16S8159182,0,2,1355,4433.0,5.972621,5.962759,6.063978,5.842376
AAACGGGTCTTACCGC-1-Human_colon_16S8159182,F72-FPIL-0-SC-1,Epithelial,"Elmentaite, 2021",F72,Fetal Healthy,16Wk,FPIL,SC,Female,10x 5' v1,...,38.484421,AAACGGGTCTTACCGC-1-Human_colon_16S8159182,0,2,3327,12475.0,5.789801,5.967800,5.726458,5.670891
AAAGATGAGATATACG-1-Human_colon_16S8159182,F72-FPIL-0-SC-1,Epithelial,"Elmentaite, 2021",F72,Fetal Healthy,16Wk,FPIL,SC,Female,10x 5' v1,...,32.269676,AAAGATGAGATATACG-1-Human_colon_16S8159182,0,2,4638,23113.0,5.524420,5.780106,5.724367,5.326749
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TGGCGCACAAGAGGCT-1-4918STDY7718977,BRC2134_10Wk_FLI_SC-EPCAMP,Epithelial,"Elmentaite, 2021",BRC2134,Fetal Healthy,10Wk,FLI,SC-EPCAMP,Male,10x 3' v1,...,24.357601,TGGCGCACAAGAGGCT-1-4918STDY7718977,0,2,1094,1916.0,5.113906,5.339201,4.959648,5.047567
TGTCCCAGTAGCGCAA-1-4918STDY7718977,BRC2134_10Wk_FLI_SC-EPCAMP,Epithelial,"Elmentaite, 2021",BRC2134,Fetal Healthy,10Wk,FLI,SC-EPCAMP,Male,10x 3' v1,...,38.328430,TGTCCCAGTAGCGCAA-1-4918STDY7718977,0,2,3934,17848.0,5.742250,5.857681,5.714561,5.606768
TTCTCAAGTGGGTATG-1-4918STDY7718977,BRC2134_10Wk_FLI_SC-EPCAMP,Epithelial,"Elmentaite, 2021",BRC2134,Fetal Healthy,10Wk,FLI,SC-EPCAMP,Male,10x 3' v1,...,34.307201,TTCTCAAGTGGGTATG-1-4918STDY7718977,0,2,2459,7096.0,5.906542,5.891854,5.792131,5.748943
